# Data Preparation
Converting the original dataset into different tables in 3rd normal form. Exporting the resulting tables as CSV.

## Importing Tools

In [99]:
import pandas as pd
import numpy as np

## Importing the Dataset

In [100]:
data = pd.read_excel("./data_original.xlsx")

## Selecting Columns of Interest & Initial Cleaning

In [101]:
data = data.drop(columns=["fix_port","source", "age_youngest", "gender", "injury_desc", "report", "notes", "mechanical", "op_error", "employee"])

In [102]:
data = data.dropna()

In [103]:
data = data[data["bus_type"]=="Amusement park"]
data = data[data["industry_sector"]=="amusement ride"]

In [104]:
data = data[data["manufacturer"] != "In-house"]

In [105]:
data = data.drop(columns=["bus_type", "industry_sector"])

In [106]:
data.head(4)

,acc_id,acc_date,acc_state,acc_city,device_category,device_type,tradename_or_generic,manufacturer,num_injured,acc_desc,category
2,1007658,2010-07-10,CA,Anaheim,water ride,Boat ride,boat ride,Arrow Dynamics,1.0,Patron stated she slipped and fell while stepp...,Load/Unload: scrape or stumble
22,1006901,2011-01-17,CA,Buena Park,coaster,Coaster - steel,inverted coaster,Bolliger & Mabillard,1.0,Ride operated normally. Patron complained of p...,Body pain (normal motion)
27,1006927,2011-01-23,CA,Anaheim,coaster,Coaster - steel,looping coaster,Intamin AG,1.0,Patron stated that she consumed a cannabis bro...,Illness or neurological symptoms
36,1006926,2011-01-29,CA,Anaheim,coaster,Coaster - steel,Mine Train,Arrow Dynamics,1.0,Patron stated she tripped over another guest a...,Load/Unload: scrape or stumble


In [108]:
data.reset_index(inplace=True)
data.drop(columns="index", inplace=True)
data.head(4)

,acc_id,acc_date,acc_state,acc_city,device_category,device_type,tradename_or_generic,manufacturer,num_injured,acc_desc,category
0,1007658,2010-07-10,CA,Anaheim,water ride,Boat ride,boat ride,Arrow Dynamics,1.0,Patron stated she slipped and fell while stepp...,Load/Unload: scrape or stumble
1,1006901,2011-01-17,CA,Buena Park,coaster,Coaster - steel,inverted coaster,Bolliger & Mabillard,1.0,Ride operated normally. Patron complained of p...,Body pain (normal motion)
2,1006927,2011-01-23,CA,Anaheim,coaster,Coaster - steel,looping coaster,Intamin AG,1.0,Patron stated that she consumed a cannabis bro...,Illness or neurological symptoms
3,1006926,2011-01-29,CA,Anaheim,coaster,Coaster - steel,Mine Train,Arrow Dynamics,1.0,Patron stated she tripped over another guest a...,Load/Unload: scrape or stumble


## Create Location Table & Relate to Data

In [109]:
location = data[["acc_city", "acc_state"]]
location = location.drop_duplicates()

In [110]:
location = location.reset_index(drop=True)
location["id"] = location.index

In [111]:
location.head(10)

,acc_city,acc_state,id
0,Anaheim,CA,0
1,Buena Park,CA,1
2,Los Angeles,CA,2
3,Valencia,CA,3
4,San Diego,CA,4
5,Riverside,CA,5
6,Carlsbad,CA,6
7,Santa Monica,CA,7
8,Vallejo,CA,8
9,Jackson,NJ,9


In [112]:
# relate tables with index
for i, row in data.iterrows():
    city = data.at[i, "acc_city"]

    for j, row in location.iterrows():
        if location.at[j, "acc_city"] == city:
            loc_id = location.at[j, "id"]
            data.at[i, "location_id"] = loc_id

In [113]:
data = data.drop(columns=["acc_city", "acc_state"])
data = data.astype({"location_id":int}, errors="raise")
data.head(5)

,acc_id,acc_date,device_category,device_type,tradename_or_generic,manufacturer,num_injured,acc_desc,category,location_id
0,1007658,2010-07-10,water ride,Boat ride,boat ride,Arrow Dynamics,1.0,Patron stated she slipped and fell while stepp...,Load/Unload: scrape or stumble,0
1,1006901,2011-01-17,coaster,Coaster - steel,inverted coaster,Bolliger & Mabillard,1.0,Ride operated normally. Patron complained of p...,Body pain (normal motion),1
2,1006927,2011-01-23,coaster,Coaster - steel,looping coaster,Intamin AG,1.0,Patron stated that she consumed a cannabis bro...,Illness or neurological symptoms,0
3,1006926,2011-01-29,coaster,Coaster - steel,Mine Train,Arrow Dynamics,1.0,Patron stated she tripped over another guest a...,Load/Unload: scrape or stumble,0
4,1008099,2011-02-06,coaster,Coaster - steel,looping coaster,Intamin AG,1.0,Patron stated she has asthma and had difficult...,Illness or neurological symptoms,0


## Create Accident_Category Table & Relate to Data

In [114]:
acc_category = data[["category"]]
acc_category = acc_category.drop_duplicates()

In [115]:
acc_category = acc_category.reset_index(drop=True)
acc_category["id"] = acc_category.index

In [116]:
acc_category.head(4)

,category,id
0,Load/Unload: scrape or stumble,0
1,Body pain (normal motion),1
2,Illness or neurological symptoms,2
3,Illness: Seizure or LOC,3


In [117]:
# relate tables with index
for i, row in data.iterrows():
    cat = data.at[i, "category"]

    for j, row in acc_category.iterrows():
        if acc_category.at[j, "category"] == cat:
            cat_id = acc_category.at[j, "id"]
            data.at[i, "category_id"] = cat_id

In [118]:
data = data.drop(columns="category")
data = data.astype({"category_id":int, "num_injured":int}, errors="raise")
data.head(5)

,acc_id,acc_date,device_category,device_type,tradename_or_generic,manufacturer,num_injured,acc_desc,location_id,category_id
0,1007658,2010-07-10,water ride,Boat ride,boat ride,Arrow Dynamics,1,Patron stated she slipped and fell while stepp...,0,0
1,1006901,2011-01-17,coaster,Coaster - steel,inverted coaster,Bolliger & Mabillard,1,Ride operated normally. Patron complained of p...,1,1
2,1006927,2011-01-23,coaster,Coaster - steel,looping coaster,Intamin AG,1,Patron stated that she consumed a cannabis bro...,0,2
3,1006926,2011-01-29,coaster,Coaster - steel,Mine Train,Arrow Dynamics,1,Patron stated she tripped over another guest a...,0,0
4,1008099,2011-02-06,coaster,Coaster - steel,looping coaster,Intamin AG,1,Patron stated she has asthma and had difficult...,0,2


## Create Category Table

In [119]:
dev_category = data[["device_category"]]
dev_category = dev_category.drop_duplicates(subset=["device_category"])

In [120]:
dev_category = dev_category.reset_index(drop=True)
dev_category["id"] = dev_category.index
dev_category

,device_category,id
0,water ride,0
1,coaster,1
2,spinning,2
3,cars & track rides,3
4,other attraction,4
5,pendulum,5
6,vertical drop,6


## Create Type Table

In [121]:
dev_type = data[["device_category", "device_type"]]

In [122]:
dev_type = dev_type.drop_duplicates(subset=["device_type"])

In [123]:
dev_type = dev_type.reset_index(drop=True)
dev_type["id"] = dev_type.index

In [124]:
dev_type.head(5)

,device_category,device_type,id
0,water ride,Boat ride,0
1,coaster,Coaster - steel,1
2,water ride,Shoot the chute,2
3,spinning,Carousel,3
4,coaster,Coaster - wooden,4


In [125]:
# relate tables with index
for i, row in dev_type.iterrows():
    dev_cat = dev_type.at[i, "device_category"]

    for j, row in dev_category.iterrows():
        if dev_category.at[j, "device_category"] == dev_cat:
            cat_id = dev_category.at[j, "id"]
            dev_type.at[i, "category_id"] = cat_id

In [126]:
dev_type = dev_type.drop(columns="device_category")
dev_type = dev_type.astype({"category_id":int}, errors="raise")
dev_type.head(5)

,device_type,id,category_id
0,Boat ride,0,0
1,Coaster - steel,1,1
2,Shoot the chute,2,0
3,Carousel,3,2
4,Coaster - wooden,4,1


## Create Device Table & Relate to Type

In [127]:
dev = data[["manufacturer", "tradename_or_generic", "device_type"]]
dev.head(5)

,manufacturer,tradename_or_generic,device_type
0,Arrow Dynamics,boat ride,Boat ride
1,Bolliger & Mabillard,inverted coaster,Coaster - steel
2,Intamin AG,looping coaster,Coaster - steel
3,Arrow Dynamics,Mine Train,Coaster - steel
4,Intamin AG,looping coaster,Coaster - steel


In [128]:
# relate tables with index
for i, row in dev.iterrows():
    type = dev.at[i, "device_type"]

    for j, row in dev_type.iterrows():
        if dev_type.at[j, "device_type"] == type:
            type_id = dev_type.at[j, "id"]
            dev.at[i, "type_id"] = type_id

/Users/edoardobianchi/DATA_SCIENZE/lib/python3.8/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/edoardobianchi/DATA_SCIENZE/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [129]:
dev = dev.drop(columns="device_type")
dev = dev.astype({"type_id":int}, errors="raise")
dev.head(5)

,manufacturer,tradename_or_generic,type_id
0,Arrow Dynamics,boat ride,0
1,Bolliger & Mabillard,inverted coaster,1
2,Intamin AG,looping coaster,1
3,Arrow Dynamics,Mine Train,1
4,Intamin AG,looping coaster,1


In [130]:
dev = dev.drop_duplicates()

In [131]:
dev = dev.reset_index(drop=True)
dev["id"] = dev.index

In [132]:
dev = dev.astype({"type_id":int}, errors="raise")

In [133]:
dev.head(5)

,manufacturer,tradename_or_generic,type_id,id
0,Arrow Dynamics,boat ride,0,0
1,Bolliger & Mabillard,inverted coaster,1,1
2,Intamin AG,looping coaster,1,2
3,Arrow Dynamics,Mine Train,1,3
4,Vekoma,shoot the chute,2,4


## Relate Device Table to Data

In [134]:
# relate tables with index
for i, row in data.iterrows():
    manuf = data.at[i, "manufacturer"]
    name = data.at[i, "tradename_or_generic"]

    for j, row in dev.iterrows():
        if (dev.at[j, "manufacturer"] == manuf) and (dev.at[j, "tradename_or_generic"] == name):
            dev_id = dev.at[j, "id"]
            data.at[i, "device_id"] = dev_id

In [135]:
data = data.drop(columns=["manufacturer", "tradename_or_generic", "device_type", "device_category"])
data = data.astype({"device_id":int}, errors="raise")

## Check Final Tables

In [136]:
data.rename(columns={"acc_id":"id", "acc_date":"date", "acc_desc":"description", "device_id":"ride_id", "category_id":"accident_category_id"}, inplace=True)
data.head(5)

,id,date,num_injured,description,location_id,accident_category_id,ride_id
0,1007658,2010-07-10,1,Patron stated she slipped and fell while stepp...,0,0,0
1,1006901,2011-01-17,1,Ride operated normally. Patron complained of p...,1,1,1
2,1006927,2011-01-23,1,Patron stated that she consumed a cannabis bro...,0,2,2
3,1006926,2011-01-29,1,Patron stated she tripped over another guest a...,0,0,3
4,1008099,2011-02-06,1,Patron stated she has asthma and had difficult...,0,2,2


In [137]:
dev.rename(columns={"tradename_or_generic":"name", "type_id":"ride_type_id"}, inplace=True)
dev.head(5)

,manufacturer,name,ride_type_id,id
0,Arrow Dynamics,boat ride,0,0
1,Bolliger & Mabillard,inverted coaster,1,1
2,Intamin AG,looping coaster,1,2
3,Arrow Dynamics,Mine Train,1,3
4,Vekoma,shoot the chute,2,4


In [138]:
dev_type.rename(columns={"device_type":"type", "category_id":"amusement_category_id"}, inplace=True)
dev_type.head(5)

,type,id,amusement_category_id
0,Boat ride,0,0
1,Coaster - steel,1,1
2,Shoot the chute,2,0
3,Carousel,3,2
4,Coaster - wooden,4,1


In [139]:
dev_category.rename(columns={"device_category":"amusement_category"}, inplace=True)
dev_category

,amusement_category,id
0,water ride,0
1,coaster,1
2,spinning,2
3,cars & track rides,3
4,other attraction,4
5,pendulum,5
6,vertical drop,6


In [140]:
location.rename(columns={"acc_city":"city", "acc_state":"state"}, inplace=True)
location.head(5)

,city,state,id
0,Anaheim,CA,0
1,Buena Park,CA,1
2,Los Angeles,CA,2
3,Valencia,CA,3
4,San Diego,CA,4


In [ ]:
acc_category.rename(columns={"category":"accident_category"}, inplace=True)
acc_category

## Exporting to CSV

In [ ]:
data.to_csv('./OUT/accident.csv', columns=["id", "date", "num_injured", "description", "location_id", "accident_category_id", "ride_id"], index=False) 
dev.to_csv('./OUT/ride.csv', columns=["id", "manufacturer", "name", "ride_type_id"], index=False) 
dev_type.to_csv('./OUT/ride_type.csv', columns=["id", "type", "amusement_category_id"], index=False) 
dev_category.to_csv('./OUT/amusement_category.csv', columns=["id", "amusement_category"], index=False) 
location.to_csv('./OUT/location.csv', columns=["id", "city", "state"], index=False) 
acc_category.to_csv('./OUT/accident_category.csv', columns=["id", "accident_category"], index=False) 